In [11]:
from pydantic import BaseModel
from typing import Literal, Optional, List
from IPython.display import display, Markdown
import json
import ollama

In [3]:
from pydantic import BaseModel
from typing import List

class EvaluatedQA(BaseModel):
    question: str
    answered: Literal['yes', 'partially', 'no']
    answer_summary: str = None
    answer_quote: str = None

class InterventionAnalysis(BaseModel):
    intervention: str
    response: str
    evaluations: List[EvaluatedQA]


In [ ]:

class QAExtractor:
    def __init__(self, model_name: str = "llama3"):
        self.model = model_name

    def extract_qa(self, intervention: str, response: str) -> InterventionAnalysis:
        messages = [
            {
                "role": "system",
                "content": """
                              You are an assistant that analyzes multi-question interventions from business or financial conference calls.

                              Given an intervention that may contain several questions, and the subsequent response:
                              - Extract each individual question clearly.
                              - Determine whether each question is answered ("yes"), partially answered ("partially"), or not answered ("no").
                              - If it is answered (fully or partially), provide a short summary and a quote from the response that supports your evaluation.
                              """
                                          },
                                          {
                "role": "user",
                "content": f"""
                              Intervention:
                              {intervention}

                              Response:
                              {response}
                """
            }
        ]

        result = json.loads(ollama.chat(model=self.model, messages=messages, format=InterventionAnalysis.model_json_schema()).message.content)
        return result

In [10]:
extractor = QAExtractor(model_name="llama3")

with open("/home/aacastro/Alejandro/ACA_MultichanelAI_2025/src/TEXT/POO/annotated/AAPL/2018/Q3.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Ahora puedes acceder a los pares por clave
for pair_key, pair_content in data.items():
    question = pair_content["Question"]
    answer = pair_content["Answer"]
    # print(f"{pair_key}:\nQ: {question}\nA: {answer}\n")
    qa_results = extractor.extract_qa(question, answer)
    print(qa_results['evaluations'])

[{'question': "First for Tim, you're on track to hit your services revenue target even earlier than planned. So how are you thinking about the next legs of services growth as you move into the next three to five years?", 'answered': 'yes', 'answer_summary': 'The company is thrilled with their current services results and feels great about their pipeline for new services.', 'answer_quote': "We feel great about our current services, but obviously we're also thrilled about our pipeline that have some new services in it as well."}, {'question': "And then for you, Luca, NAN prices are falling this year. Services mix is rising. Those should both positively influence gross margins and yet We're seeing gross margins sort of hang out here at 38%. What are the offsetting headwinds, and is it possible that we could see the tailwinds start to overpower those headwinds in the next couple of quarters and see gross margins drift higher?", 'answered': 'partially', 'answer_summary': "The company mentio

In [13]:
for pair_key, pair_content in data.items():
    question = pair_content["Question"]
    answer = pair_content["Answer"]
    qa_results = extractor.extract_qa(question, answer)
    evaluations = qa_results.get("evaluations", [])

    # Formateo bonito usando Markdown
    md_output = f"---\n### 🔹 {pair_key}\n"
    md_output += f"**📌 Question:**\n{question}\n\n"
    md_output += f"**📌 Answer:**\n{answer}\n\n"

    if evaluations:
        for eval_idx, eval_entry in enumerate(evaluations, start=1):
            md_output += f"**📊 Evaluation #{eval_idx}:**\n"
            md_output += f"- **Question evaluated:** {eval_entry.get('question', '')}\n"
            md_output += f"- **Answered:** {eval_entry.get('answered', '')}\n"
            md_output += f"- **Answer summary:** {eval_entry.get('answer_summary', '')}\n"
            md_output += f"- **Answer quote:** _{eval_entry.get('answer_quote', '')}_\n\n"
    else:
        md_output += "**⚠️ No evaluations found.**\n"

    display(Markdown(md_output))

---
### 🔹 pair_1
**📌 Question:**
Thank you and congrats on the quarter. I'll ask both my questions up front. First for Tim, you're on track to hit your services revenue target even earlier than planned. So how are you thinking about the next legs of services growth as you move into the next three to five years? And then for you, Luca, NAN prices are falling this year. Services mix is rising. Those should both positively influence gross margins and yet We're seeing gross margins sort of hang out here at 38%. What are the offsetting headwinds, and is it possible that we could see the tailwinds start to overpower those headwinds in the next couple of quarters and see gross margins drift higher? Thank you.

**📌 Answer:**
Yeah, Katie, thanks for your questions, Tim. On the services side, we're thrilled with the results. They were very broad-based. We had double-digit services growth in all of the geographic segment. And the App Store, AppleCare, Apple Music, Cloud Services, Apple Pay all set new June quarter records. And, of course, subscriptions have now passed over 300 million, as I had mentioned before. And so we couldn't be happier with how things are going. In terms of the next leg of this, given the momentum that we're seeing across the board, we feel great about our current services, but obviously we're also thrilled about our pipeline that have some new services in it as well. And so with the combination of these, we feel great about

**📊 Evaluation #1:**
- **Question evaluated:** how are you thinking about the next legs of services growth as you move into the next three to five years?
- **Answered:** yes
- **Answer summary:** The company is thrilled with their current services and has a strong pipeline for new services, feeling great about their future growth.
- **Answer quote:** _given the momentum that we're seeing across the board, we feel great about our current services, but obviously we're also thrilled about our pipeline that have some new services in it as well_

**📊 Evaluation #2:**
- **Question evaluated:** What are the offsetting headwinds, and is it possible that we could see the tailwinds start to overpower those headwinds in the next couple of quarters and see gross margins drift higher?
- **Answered:** no
- **Answer summary:** The company did not directly answer this question.
- **Answer quote:** __



---
### 🔹 pair_2
**📌 Question:**
Thank you very much for taking my question. Tim, can you talk a bit about trends within your iPhone sales? ASPs were above expectations, you know, and now that you've had – And that was clearly better than some of the comments from some of your competitors. Now that you've had about nine months of experience with a, you know, high-end, fully-featured phone, can you talk a bit about what you think customers want, what the elasticity of demand is, and, you know, how you're sort of thinking about your competitive position? And then I have a follow-up.

**📌 Answer:**
Yeah. Shannon, we feel great about the results on iPhone, up 20%. And if you look, for the cycle, and by the cycle I mean Q1, Q2 and Q3, we've had on an average weekly basis growth in units of sort of mid-single digit and ASP growth of double digit. And so if you look at iPhone X in particular, it's the most innovative smartphone on the market. We priced it at a level that represented the value of it. And we're really, we could not be happier that it has been the top selling iPhone since the launch. And so we feel terrific about iPhone 10. If you look at the sort of the top of our line together, and by that I mean the iPhone 10, the 8, and the 8 Plus, they are growing very nicely, as you can probably tell from looking at the ASP, and we couldn't be happier with how that's gone. And so I think in this cycle we've learned that customers want innovative products, and we sort of already knew that in other cycles and other points in time, but it just – puts an explanation point by that, I believe, with looking at the results. At the unit level, the iPhone SE had a difficult comp to the year-ago quarter, and so when we changed some of the configurations, the memory configurations in the year-ago quarter, if you look at it on a geographic basis, The top three selling phones in urban China were iPhone, where iPhone 10 was number one and has been for a couple of quarters. And iPhones make up three of the top five smartphones in the US, UK and Japan. And so, you know, it's difficult sometimes to get a read over exactly what's happening in the market, but given the industry numbers that we've seen, it's clear that we picked up global market share and picked up market share in several countries, not only iPhone, but iPad as well.

**📊 Evaluation #1:**
- **Question evaluated:** can you talk a bit about trends within your iPhone sales? ASPs were above expectations, you know, and now that you've had – And that was clearly better than some of the comments from some of your competitors. Now that you've had about nine months of experience with a, you know, high-end, fully-featured phone, can you talk a bit about what you think customers want, what the elasticity of demand is, and, you know, how you're sort of thinking about your competitive position?
- **Answered:** yes
- **Answer summary:** Tim talked about the positive trends in iPhone sales, with ASPs above expectations. He also highlighted that customers want innovative products.
- **Answer quote:** _And so we feel terrific about iPhone 10. If you look at the sort of the top of our line together, and by that I mean the iPhone 10, the 8, and the 8 Plus, they are growing very nicely, as you can probably tell from looking at the ASP, and we couldn't be happier with how that's gone._



---
### 🔹 pair_3
**📌 Question:**
And then can you talk a bit about China, greater China, up 19% year over year during the quarter, I believe, You know, obviously iPhone doing well, but, you know, some concern that maybe some of what's going on in the trade world might have impacted. It doesn't seem like that. So I'm just curious as to what you're seeing in China and how you're thinking about it as you look forward. Thank you.

**📌 Answer:**
Yeah, it's a good question. Thank you. This is the fourth consecutive quarter that we've had double-digit growth in greater China. I mentioned how iPhone X and sort of the iPhones are selling. We did pick up share in iPhone and iPad. But if you look more holistically at our complete line, we had double-digit growth from services to iPad to iPhone and to our other product category, which the watch did extremely well. And so there are lots of good things happening there in terms of the – The tariffs themselves, maybe I could sort of take a step back because I'm sure that a lot of some people have questions on this. Our view on tariffs is that they show up as a tax on the consumer and wind up resulting in lower economic growth and sometimes can bring about significant risk of unintended consequences. That said, the trade relationships and agreements that the U.S. has between the U.S. and other major economies are very complex, and it's clear that several are in need of modernizing. But we think that in the vast majority of situations that tariffs are not the approach to doing that. And so we're sort of encouraging dialogue and so forth. In terms of the tariffs that have been imposed or have exited the comment period, I think there's one that's exiting today, there had been three of those, and maybe I could walk through those briefly just to make sure everybody's on the same page. The first was the U.S. imposed a tariff on steel and aluminum that was many, many different countries. That started, I believe, at the beginning of June. There have been two other tariffs that have totaled about $50 billion of goods from China that have either been implemented or exiting the comment period in this month. I think the latest one is today. If you look at those three tariffs, none of our products were directly affected by the tariffs. There is a fourth tariff. which includes goods valued at $200 billion, also focused on goods that are imported from China. That one is out for public comment. Probably like everyone else, we're evaluating that one, and we'll be sharing our views of it with the administration and so forth before the comment period for that one ends. It's actually a tedious process in going through it, because you not only have to analyze the revenue products, which are a bit more straightforward to analyze, but you also have to analyze the purchases that you're making through other companies that are not related to revenue. Maybe they're related to data centers and this sort of thing. And so we're going through that now and we'll be sharing our results. later on those and feeding back public comment. Of course, the risk associated with more of a macroeconomic issue, such as an economic slowdown in one or more countries, or currency fluctuations that are related to tariffs is very difficult to quantify. And so that, and we're not even trying to quantify that, to be clear about it. All of this said, We're optimistic, as I've been the whole time, that this will get sorted out, because there's an inescapable mutuality between the US and China that sort of serves as a magnet to bring both countries together. Each country can only prosper if the other does, and of course the world needs both US and China to prosper for the world to do well. That said, I can't predict the future, but I am optimistic that the countries will get through this, and we are hoping that calm heads prevail.

**📊 Evaluation #1:**
- **Question evaluated:** What are you seeing in China?
- **Answered:** partially
- **Answer summary:** The speaker mentions double-digit growth in greater China for the fourth consecutive quarter. They also highlight the success of iPhone, iPad, and watch sales.
- **Answer quote:** _This is the fourth consecutive quarter that we've had double-digit growth in greater China._

**📊 Evaluation #2:**
- **Question evaluated:** How are you thinking about China as you look forward?
- **Answered:** partially
- **Answer summary:** The speaker expresses optimism for resolving trade issues and improving US-China relations.
- **Answer quote:** _We're optimistic, as I've been the whole time, that this will get sorted out, because there's an inescapable mutuality between the US and China that sort of serves as a magnet to bring both countries together._

**📊 Evaluation #3:**
- **Question evaluated:** What's your view on tariffs?
- **Answered:** partially
- **Answer summary:** The speaker views tariffs as a tax on consumers, leading to lower economic growth and unintended consequences. They also acknowledge the complexity of trade relationships.
- **Answer quote:** _Our view on tariffs is that they show up as a tax on the consumer and wind up resulting in lower economic growth and sometimes can bring about significant risk of unintended consequences._



---
### 🔹 pair_4
**📌 Question:**
Tim, I'm wondering if you could talk a little bit about the multi-year partnership with Oprah Winfrey and what that says about your original content strategy and also Apple Music, if you can give us a little more colors or an update maybe around paid subscribers or total subscribers around Apple Music and how you feel that's rolled out.

**📌 Answer:**
Yeah, sure, Brian. Thanks for the question. We're very excited to work with Oprah. We think that her incomparable ability and talent to connect with audiences around the world, there's sort of no match. And we think that we can do some great in original content together. And so we could not be happier in working with Oprah. As you know, we hired two highly respected television executives last year and they have been here now for several months and have been working on a project that we're not really ready to share all the details of it yet. But I couldn't be more excited about what's going on there and we've got great talent in the area that we've sourced from different places and feel really good about what we will eventually offer. In terms of the key catalyst and the changes, the cord cutting in our view is only going to accelerate and probably accelerate at a much faster rate than is widely thought. We're seeing the things that we have on the periphery of this, like Apple TV units and revenue grew by very strong double digits in Q3. As I mentioned in my opening comments, we're seeing different providers pick up the Apple TV and use it as their box to go to market with their subscription service. There are within the 300 million plus paid subscriptions, some of these are third party video subscriptions, and we see the growth that is going on there. It's like 100% year over year. And so all the things from a all the forcing functions here from the outside all point to dramatic changes speeding up in the content industry and so we're really happy to be working on something but just not ready to talk about it in depth today. In terms of Apple Music, we're well over 50 million listeners now when you add our paid subscribers and the folks in the trial. And so we're moving along at a very, very good rate. It appears to us, or what we've been told, is that we took the leadership position in North America during the quarter. And we have leadership position in Japan. And so in some of the markets that we've been in for a long period of time, we're doing quite well. But really, the key thing in music is not the competition between companies that are providing music. The real challenge is to grow the market. Because if you add everyone up that's providing subscription music today or streaming music, it's outside of China, it's less than 200 million probably around the world. And so it does seem to me there's an extraordinary opportunity in that business to grow the market well. And I think if we put our emphasis there, which we're doing, That will be a beneficiary of that, as other people will as well. But I like where we are. Our revenues on Apple Music grew over 50%, as I mentioned earlier, during the quarter. And so some really, really strong, strong results.

**📊 Evaluation #1:**
- **Question evaluated:** What does the multi-year partnership with Oprah Winfrey say about your original content strategy?
- **Answered:** yes
- **Answer summary:** The partnership with Oprah Winfrey is a great opportunity for the company to create original content together. The speaker believes that her talent and ability to connect with audiences will be a great asset in this collaboration.
- **Answer quote:** _We think that her incomparable ability and talent to connect with audiences around the world, there's sort of no match._

**📊 Evaluation #2:**
- **Question evaluated:** Can you give an update on paid subscribers or total subscribers around Apple Music?
- **Answered:** yes
- **Answer summary:** The company is well over 50 million listeners now when adding both paid subscribers and those in trial. The speaker believes that this growth is a sign of the company's strong performance.
- **Answer quote:** _We're well over 50 million listeners now when you add our paid subscribers and the folks in the trial._

**📊 Evaluation #3:**
- **Question evaluated:** How do you feel about how Apple Music has rolled out?
- **Answered:** yes
- **Answer summary:** The speaker is pleased with the progress of Apple Music, mentioning that it took a leadership position in North America during the quarter.
- **Answer quote:** _It appears to us, or what we've been told, is that we took the leadership position in North America during the quarter._

**📊 Evaluation #4:**
- **Question evaluated:** What does the partnership say about your original content strategy?
- **Answered:** yes
- **Answer summary:** The speaker believes that the partnership with Oprah Winfrey will be a great opportunity to create original content together.
- **Answer quote:** _We think that we can do some great in original content together._

**📊 Evaluation #5:**
- **Question evaluated:** How does Apple Music's growth compare to others?
- **Answered:** no
- **Answer summary:** The speaker didn't provide direct comparison with other companies, but mentioned the market size of subscription music being less than 200 million outside of China.
- **Answer quote:** _Because if you add everyone up that's providing subscription music today or streaming music, it's outside of China, it's less than 200 million probably around the world._

**📊 Evaluation #6:**
- **Question evaluated:** How do you see Apple Music growing?
- **Answered:** yes
- **Answer summary:** The speaker believes that there is an extraordinary opportunity to grow the market for subscription music, and that the company is well-positioned to take advantage of this.
- **Answer quote:** _I think if we put our emphasis there, which we're doing, That will be a beneficiary of that, as other people will as well._

**📊 Evaluation #7:**
- **Question evaluated:** What are your thoughts on Apple Music's revenue growth?
- **Answered:** yes
- **Answer summary:** The company saw over 50% revenue growth in the quarter.
- **Answer quote:** _Our revenues on Apple Music grew over 50%, as I mentioned earlier, during the quarter._



---
### 🔹 pair_5
**📌 Question:**
Yes, thank you. I have one for Luca and one for Tim, please. Luca, I'm wondering, as we think about modeling Q4, iPhone ASPs are typically up sequentially about 2% to 4%, sort of low single digits. Perhaps you can help us think through how we should be thinking about Q4. I know you provided some commentary last quarter on how we should be thinking about Q3 ASPs.

**📌 Answer:**
Tony, as you know, we do not provide guidance for either units or ASPs for any product category, but of course, you know, we provide the guidance on revenue and the guidance The range implies growth of 16% to 19%. We expect the growth to come from strong growth from iPhone, from services, and from wearables, which has been a bit of our pattern during the course of the year. On iPhone ASP, the only thing that I will point out is that obviously we're exiting the June quarter at a significantly higher level than in the past, and so that I think it's important to to keep in mind as we move into the September quarter. It's important to keep in mind the type of revenue growth that we've implied in our guidance.

**📊 Evaluation #1:**
- **Question evaluated:** As we think about modeling Q4, iPhone ASPs are typically up sequentially about 2% to 4%, sort of low single digits.
- **Answered:** partially
- **Answer summary:** The answer is partially because the executive provides guidance on revenue growth (16-19%) but does not provide specific guidance on iPhone ASP. The quote supporting this evaluation is: 'we do not provide guidance for either units or ASPs for any product category'
- **Answer quote:** _'We do not provide guidance for either units or ASPs for any product category'_



---
### 🔹 pair_6
**📌 Question:**
Tim, I was wondering if you could just comment a little bit about the health of the smartphone market. Apple's smartphone, iPhone units have been relatively flat for four years, and I think you've probably been a share gainer during the period. which would suggest at least at the high end a market that is perhaps flat to down. And I'm wondering if you can comment on, A, whether you believe that and what you think might be happening with replacement cycles, and specifically also what impact, if any, you've seen from wider availability and less expensive replacement batteries for iPhones.

**📌 Answer:**
I think the smartphone market is very healthy.

**📊 Evaluation #1:**
- **Question evaluated:** What do you believe about the health of the smartphone market?
- **Answered:** yes
- **Answer summary:** According to Tim, he believes the smartphone market is 'very healthy'. This suggests that despite iPhone unit sales being relatively flat for four years, there are other factors contributing to the overall health of the market.
- **Answer quote:** _I think the smartphone market is very healthy._



---
### 🔹 pair_7
**📌 Question:**
Okay, super. So I'd like to focus on product roadmap and strategy. There is a war going on for the home, the connected home of the Internet of Things, and with two products, the HomePod and Apple TV in the home. My question is strategically, how do you feel about the importance of being in the home and whether it threatens your dominance outside the home with your core business and the mobile devices if you sort of lose that battle. I'm just trying to figure out strategically when you think about where the puck's going, how important is it for you to have a beachhead in the home as well as out of home?

**📌 Answer:**
I think the home business, Laura, is bigger than the HomePod and Apple TV. They're both important products, clearly. But everybody has their iPhone at home as well and everybody has their Mac at home and everybody has their iPad at home. And so in terms of the Siri access point, as you can tell from the 100 billion number I quoted in the script, there's an extraordinary amount of usage of these products that are used to perform home-related functions. You know, I do that every day with controlling all my home automation and so on and so forth. Part of that is on HomePod, but part of it is with the Apple Watch and the iPhone and the iPad. And so I think home is important. Home is important. Work is important. The movement between the two are important. Health is important. The smartphone has become the repository that goes across the whole of your life, not something that is just meant for a portion of it. And so I think all of those are important, and we're focused on all of them.

**📊 Evaluation #1:**
- **Question evaluated:** how do you feel about the importance of being in the home and whether it threatens your dominance outside the home with your core business and the mobile devices if you sort of lose that battle. I'm just trying to figure out strategically when you think about where the puck's going, how important is it for you to have a beachhead in the home as well as out of home?
- **Answered:** yes
- **Answer summary:** The speaker believes being in the home is bigger than just having HomePod and Apple TV. They emphasize that everyone has their iPhone, Mac, and iPad at home, which highlights the importance of Siri access points for performing home-related functions. The speaker also emphasizes the importance of home, work, movement between the two, health, and the smartphone being a repository across one's life.
- **Answer quote:** _I think home is important. Home is important. Work is important. The movement between the two are important. Health is important._



---
### 🔹 pair_8
**📌 Question:**
But do they attract a new person to the ecosystem? Or does the person have to have an iPhone first?

**📌 Answer:**
It is – well, but on your point, though, it is clear from communications I've had with users that some of them were attracted to iPhone because of the Apple Watch. And so the Apple Watch led them to the iPhone. The reverse of that is also true, is that somebody got the iPhone and then decided, you know, I really want something to – to coach me in fitness and to curate some of the communications and so forth, like the watch desk as well. And so it's not always a linear path. I see these things as being somewhat fluid and different for each user.

**📊 Evaluation #1:**
- **Question evaluated:** But do they attract a new person to the ecosystem? Or does the person have to have an iPhone first?
- **Answered:** partially
- **Answer summary:** The response does not directly answer whether Apple Watch attracts new people to the ecosystem or if users need an iPhone first. However, it partially answers by highlighting that some users were attracted to iPhone because of the Apple Watch, and vice versa.
- **Answer quote:** _some of them were attracted to iPhone because of the Apple Watch._

